In [ ]:
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 50, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 50 (delta 18), reused 20 (delta 5), pack-reused 0
Unpacking objects: 100% (50/50), done.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
     |████████████████████████████████| 19.4MB 7.3MB/s 
     |████████████████████████████████| 92kB 9.6MB/s 
     |████████████████████████████████| 2.3MB 41.2MB/s 
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2020-02-05 04:07:34--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz


In [ ]:
import pandas as pd

import torch
# import mymodel
import torch.nn as nn
import torch.nn.functional as F 
from torch.autograd import Variable 
import pickle 

import MeCab
#from mecab_split import mecabsplit
tagger = MeCab.Tagger()
#tagger = MeCab.Tagger('-d /usr/local/lib/mecab/dic/mecab-ko-dic')

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


## Data setting

In [ ]:
df=pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/college_faq/college_faq_data.csv',encoding='utf-8')

In [ ]:
df.tail()

,category,question,answer
335,기타,학술적 글쓰기 학습지원을 받고 싶은데 어디로 문의해야 하나요?,Open & Smart Education 센터 글쓰기교실에서는 학술적 글쓰기 개별 ...
336,자원봉사,"학사포털에 자원봉사를 입력하였는데, 저장이 안돼요",학사포탈(portal.yonsei.ac.kr)에서 자원봉사 정보 입력 후 저장버튼을...
337,자원봉사,인증서를 어디서 뽑나요?,학사포탈(portal.yonsei.ac.kr) -> 자원봉사 -> 자원봉사 출력\r...
338,자원봉사,자원봉사활동을 하고 난 뒤 자원봉사 승인을 받으려면 어떻게 해야 하나요?,"1) 자원봉사활동 수행하고 나서, 자원봉사 확인서를 받으세요\r\n2) 학사포탈(p..."
339,자원봉사,사회봉사과목을 수강철회하고 싶습니다.,"사회봉사과목은 수강철회가 되지 않는 과목이니, 수강신청할 때 유의해서 신청하기 바랍..."


In [ ]:
df.category.value_counts()

등록           73
생활관          23
증명서          23
수강신청         20
휴복학          20
복수전공/연계전공    20
계절제수업        17
기타           17
장학금          15
재수강          14
성적/출결        12
캠퍼스견학        12
졸업           11
도서관          10
소속변경         10
ID카드          9
건강센터          9
YSCEC         8
교환학생          7
자원봉사          4
성폭력/성평등       3
장애학생지원        2
교원자격증         1
Name: category, dtype: int64

In [ ]:
subset_category=['등록','생활관','수강신청','휴복학','장학금']

df=df[df.category.isin(subset_category)]

In [ ]:
def mecabsplit(mecab_tagger,inputs, pos):
    r=[]
    inputs = mecab_tagger.parse(inputs)
    t = inputs.split('\n')[:-2]
    for i in t:
        field = i.split('\t')
        if field[1].split(',')[-1].strip() is not '*':
            r.extend( [ (x.split('/')[0],x.split('/')[1]) for x in field[1].split(',')[-1].split('+') ] )
        else:
            r.append( (field[0],field[1].split(',')[0]) )
    if pos:
        return r
    else:
        return [ x[0] for x in r ]
    return r

In [ ]:
content_vocab = {'unk':0}
intent_vocab={}
intent_list=[]

data_intent=''
intent_idx=0
vocab_idx=1

for line in df.category.unique():
    intent=line
    if intent not in intent_vocab:
        intent_vocab[intent]=intent_idx
        intent_list.append(intent)
        intent_idx +=1

for line in list(df.question):
    line = mecabsplit(tagger,line,False)
    for it in line:
        if it not in content_vocab:
            content_vocab[it] = vocab_idx # it : 단어
            vocab_idx +=1

In [ ]:
print(content_vocab)
print(intent_vocab)
print(intent_list)

{'unk': 0, '대학': 1, '원': 2, '수료': 3, '생': 4, '(': 5, '당': 6, '학기': 7, '논문': 8, '제출': 9, '예정': 10, '자': 11, ')': 12, '의': 13, '경우': 14, '등록': 15, '금': 16, '및': 17, '생활': 18, '비': 19, '대출': 20, '이': 21, '가능': 22, '하': 23, 'ᆫ가요': 24, '?': 25, '학위': 26, '과정': 27, '은': 28, '4': 29, '년': 30, '제': 31, '9': 32, ',': 33, '2': 34, '제의': 35, '5': 36, '등': 37, '과': 38, '같': 39, '초과': 40, '학생': 41, '도': 42, '학자': 43, '을': 44, '받': 45, '수': 46, '있': 47, '나요': 48, '평점': 49, '백': 50, '분위': 51, '환산': 52, '점수': 53, '로': 54, '79': 55, '.': 56, '7': 57, '점': 58, 'ᄇ니다': 59, '들': 60, '든': 61, '재': 62, '입학': 63, '이나': 64, '편입': 65, '재학': 66, '군': 67, '신입': 68, '고지': 69, '서': 70, '출력': 71, '어디': 72, '에서': 73, 'ᆯ': 74, '왜': 75, '안': 76, '되': 77, '기간': 78, '이전': 79, '에': 80, '미리': 81, '를': 82, '고': 83, '싶': 84, '어요': 85, '복학': 86, '신청': 87, 'ᆫ데': 88, '았': 89, '는데': 90, '금액': 91, '이상': 92, '게': 93, '나오': 94, 'ㅏ요': 95, '다음': 96, '소속': 97, '변경': 98, '가': 99, '어떻게': 100, '는지': 101, '알': 102, '계열': 103, '별': 104, 'ᆫ

## Modeling

In [ ]:
class CNN_Text(nn.Module):
    
    def __init__(self, embed_num, class_num, ):
        super(CNN_Text, self).__init__()
        
        V = embed_num
        D = 100 #args.embed_dim
        C = class_num
        Ci = 1
        Co = 20 #args.kernel_num
        Ks = [1,2,3]

        self.embed = nn.Embedding(V, D)  # 입력문장 내 443개 단어의 word vector에 random 초기값

        self.convs1 = nn.ModuleList([nn.Conv2d(Ci, Co, (K, D)) for K in Ks])

        self.dropout = nn.Dropout(0.2)
        self.fc1 = nn.Linear(len(Ks)*Co, C)  # (60, 5)

    def forward(self, x):
        # x : 패턴 파일의 한 줄(한 문장)
        #print (x.size())
        x = self.embed(x)  # (N, W, D) (1문장, 패턴의 단어수, 100차원)
        # 문장 x의 단어 벡터값 가져옴
        #print (x.size())
        x = x.unsqueeze(1)  # (N, Ci, W, D) # Ci를 추가하여 unsqueeze (Text는 채널 1개, 이미지는 채널 3개)
        #print (x.size())
        
        x = [F.relu(conv(x)).squeeze(3) for conv in self.convs1]  # [(N, Co, W), ...]*len(Ks)
        # 컨볼루션 하는 부분. 입력문 별로 호출. convs1에 필터 3개 있음 (각 1, 2, 3개 단어)
        # 입력문장에 필터 3개를 적용
        #print(len(x))
        #print(x[0].size())
        #print(x[1].size())
        #print(x[2].size())
        
        x = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in x]  # [(N, Co), ...]*len(Ks)
        # max pooling해서 60개 값이 나옴
        
        x = torch.cat(x, 1)

        x = self.dropout(x)  # (N, len(Ks)*Co)
        logit = self.fc1(x)  # (N, C) 
        # 결과값(logit)은 실수값 2개 출력, 이 중 큰값을 가진 쪽이 분류된 클래스
        #print(logit)
        return logit


In [ ]:
# cnn = mymodel.CNN_Text(vocab_idx,intent_idx)
cnn = CNN_Text(vocab_idx,intent_idx)
print(vocab_idx, intent_idx) # 파일에 나타난 단어수, 의도 갯수(클래스)
optimizer = torch.optim.Adam(cnn.parameters())
cnn.train()    # train mode로 준비

443 5


CNN_Text(
  (embed): Embedding(443, 100)
  (convs1): ModuleList(
    (0): Conv2d(1, 20, kernel_size=(1, 100), stride=(1, 1))
    (1): Conv2d(1, 20, kernel_size=(2, 100), stride=(1, 1))
    (2): Conv2d(1, 20, kernel_size=(3, 100), stride=(1, 1))
  )
  (dropout): Dropout(p=0.2, inplace=False)
  (fc1): Linear(in_features=60, out_features=5, bias=True)
)

## Training

In [ ]:
epoch = 10
for e in range(epoch):
    totalloss = 0
    
    for line in df.values:
        target=Variable(torch.LongTensor([intent_vocab[line[0]]]))

        optimizer.zero_grad()

        cont = []
        line = mecabsplit(tagger,line[1],False)
        
        for it in line:
            cont.append(content_vocab[it])
        
        
        cont = Variable(torch.LongTensor(cont)).view(1,-1)
        pred = cnn(cont)
        
        loss = F.cross_entropy(pred,target)
        totalloss += float(loss.data)
        loss.backward()  # 필터, 임베딩벡터 값, FC 레이어의 matrix 등 모든 값을 loss를 줄이는 방향으로 계산
        optimizer.step() # 벡터값을 변경함
    print (totalloss)

233.29798401612788
176.61124399583787
109.8286725115031
63.724793850036804
39.22011807578383
20.501023083226755
15.88645585764607
7.562997944527524
8.459266963636765
5.288595629070187


## Response

In [ ]:
response = df.answer

In [ ]:
with open('/content/gdrive/My Drive/Colab Notebooks/college_faq/college_faq_response.txt', 'r', encoding='utf-8') as response_file:
    response = [row.split('\t')[1] for row in response_file.readlines()[1:]]

## Test

In [ ]:
test=[]
for i in intent_list:
    subset_df=df.query('category=="{}"'.format(i))
    test.append(subset_df.question.values[0])

In [ ]:
test

['대학원 수료생(당 학기 논문제출 예정자)의 경우 등록금 및 생활비 대출이 가능한가요?',
 '동록금을 납부한 뒤 휴학을 하면 나중에 복학할 때 등록금을 납부하지 않아도 되나요?',
 '2, 3차 복학 후 수강신청시 불이익이 있나요?',
 '한국장학재단 학자금 대출 시 특별 추천은 어떤 경우에 받을 수 있나요?',
 '생활관과 신촌역을 연결하는 교통편이 있나요?']

In [ ]:
## your test input
test=['1학기 수강신청 기간 언제인가요?']

In [ ]:
for idx in list(test):
    line = mecabsplit(tagger,idx,False)
   
    cont = []
    for it in line:
        if it in content_vocab:
            cont.append(content_vocab[it])  # cont에는 입력문에 나타난 단어들의 index 저장됨
        else:
            cont.append(content_vocab['unk'])
    
    cont = Variable(torch.LongTensor(cont)).view(1,-1)
    pred = cnn(cont)
    v,i = torch.max(pred,1) # i: 가장 확률 높은 클래스의 인덱스
    
    print('input : ',line)
    print('intent : ',intent_list[int(i)])
    print('answer : ',response[int(i)],'\n')

input :  ['1', '학기', '수강', '신청', '기간', '언제', '이', 'ᆫ가요', '?']
intent :  수강신청
answer :  수강신청 관련한 문의에 대한 답변입니다.
 

